# Tutorial

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
import torch
import wandb

import pytorch_lightning as pl

# from codecarbon import EmissionsTracker
from pathlib import Path
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, TQDMProgressBar
from pytorch_lightning.loggers import WandbLogger
from retrofit.data import RetroDataset
from retrofit.model import RetroFitModelWrapper
from transformers import AutoTokenizer, EncoderDecoderModel

In [ ]:
column = "whole_func_string"
encoder_name = "microsoft/codebert-base"
decoder_name = "gpt2"
encoder_tokenizer = AutoTokenizer.from_pretrained(encoder_name)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_name)
decoder_tokenizer.pad_token = decoder_tokenizer.eos_token
retro_ds = RetroDataset(
    "code_search_net",
    "flax-sentence-embeddings/st-codesearch-distilroberta-base",
    encoder_tokenizer,
    decoder_tokenizer,
    dataset_config="python",
    column=column,
    batch_size=4,
    k=2,
    n_perc=2
)

model = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_name, decoder_name)
model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.pad_token_id = decoder_tokenizer.pad_token_id
model.config.decoder_max_length = 512
retro_model = RetroFitModelWrapper(
    model,
    weight_decay=0.1,
    lr=5e-4,
    freeze_decoder=True
)

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.10.ln_cross_attn.weight', 'h.5.crossattention.c_attn.weight', 'h.7.crossattention.c_proj.bias', 'h.8.crossattention.q_attn.weight', 'h.9.crossattention.c_proj.weight', 'h.6.crossattention.bias', 'h.7.crossattention.c_proj.weight', 'h.0.crossattention.c_proj.bias', 'h.7.crossattention.masked_bias', 'h.11.crossattention.bias', 'h.3.crossattention.bias', 'h.6.crossattention.c_proj.bias', 'h.0.ln_cross_attn.weight', 'h.2.crossattention.c_proj.weight', 'h.10.crossattention.c_proj.bias', 'h.9.crossattention.q_attn.weight', 'h.1.crossattention.q_attn.weight', 'h.9.crossattention.bias', 'h.11.crossattention.q_attn.weight', 'h.5.crossattention.q_attn.weight', 'h.8.crossattention.c_proj.bias', 'h.2.crossattention.bias', 'h.7.ln_cross_attn.weight', 'h.11.ln_cross_attn.weight', 'h.5.crossattention.bias', 'h.9.crossattention.masked_bias', 'h.6.crossattention.c_attn.weight', 'h.9.ln_

In [ ]:
def train_model(model, data_module, num_epochs, output_dir, gradient_accumulation_steps=1):
    """
    Train a model with a given training data loader, validation data loader,
    optimizer, scheduler, loss function, metrics, and callbacks.

    Args:
        model (pl.LightningModule): The model to train.
        data_module (pl.LightningDataModule): The data module to use for training.
        num_epochs (int): The number of epochs to train for.
        output_dir (pathlib.Path): The directory to save the model to.
        name (str): The name of the model.
    Returns:
        best_model_path (str): The path to the best model's checkpoint.
    """
    # pl.seed_everything(115, workers=True)
    wandb_logger = WandbLogger(project="Retrofit")
    # saves a file like: my/path/sample-mnist-epoch=02-val_loss=0.32.ckpt
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=str(output_dir / "checkpoints"),
        filename="retrofit-{epoch:02d}-{val_loss:.2f}",
        save_top_k=5,
        mode="min",
    )
    trainer = pl.Trainer(
        logger=wandb_logger,
        default_root_dir=str(output_dir / "checkpoints"),
        gpus=torch.cuda.device_count(),
        max_epochs=num_epochs,
        limit_train_batches=0.1,
        limit_val_batches=0.1,
        precision=16,
        accumulate_grad_batches=gradient_accumulation_steps,
        callbacks=[
            checkpoint_callback,
            EarlyStopping(monitor="val_loss"),
            TQDMProgressBar(refresh_rate=1),
        ],
    )
    # tracker = EmissionsTracker(output_dir=output_dir.parent.parent, project_name=wandb_logger.name)

    # train the model and track emissions
    # tracker.start()
    trainer.fit(model, data_module)
    # tracker.stop()

    # save the best model to wandb
    best_model_path = checkpoint_callback.best_model_path
    if best_model_path is not None:
        wandb.save(best_model_path)

    # # save the emissions csv file
    # wandb.save(str(output_dir.parent.parent / "emissions.csv"))

    return best_model_path

In [ ]:
num_epochs = 5
out_dir = Path("/workspace/retrofit/data/output/")
best_model_path = train_model(
    retro_model,
    retro_ds,
    num_epochs=num_epochs,
    output_dir=out_dir / "model",
    gradient_accumulation_steps=4,
)
# model = RetroFitModel.load_from_checkpoint(best_model_path)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Reusing dataset code_search_net (/root/.cache/huggingface/datasets/code_search_net/python/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)
Reusing dataset code_search_net (/root/.cache/huggingface/datasets/code_search_net/python/1.0.0/80a244ab541c6b2125350b764dc5c2b715f65f00de7a56107a28915fac173a27)


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/8244 [00:00<?, ?ex/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


In [ ]:
model = RetroFitModelWrapper.load_from_checkpoint(best_model_path)